In [1]:
%reload_kedro

2021-10-07 19:04:46,630 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-10-07 19:04:46,679 - root - INFO - ** Kedro project spaceflightsTutorial
2021-10-07 19:04:46,681 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-10-07 19:04:46,690 - root - INFO - Registered line magic `run_viz`


In [2]:
context.catalog

In [3]:
context.pipeline

/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/kedro/framework/context/context.py:59: DeprecationWarning: Accessing pipeline via the context will be deprecated in Kedro 0.18.0.
  warn(


Pipeline([
Node(preprocess_shuttles, 'shuttles', 'preprocessed_shuttles', 'preprocess_shuttles_node'),
Node(preprocess_companies, 'companies', 'preprocessed_companies', 'preprocess_companies_node'),
Node(create_model_input_table, ['preprocessed_shuttles', 'preprocessed_companies', 'reviews'], 'model_input_table', 'create_model_input_table_node'),
Node(split_data, ['model_input_table', 'parameters'], ['X_train', 'X_test', 'y_train', 'y_test'], 'split_data_node'),
Node(train_model, ['X_train', 'y_train'], 'regressor', 'train_model_node'),
Node(train_model_RF, ['X_train', 'y_train'], 'RF_regressor', 'train_model_RF_node'),
Node(evaluate_model_RF, ['RF_regressor', 'X_test', 'y_test'], None, 'evaluate_model_RF_node'),
Node(evaluate_model, ['regressor', 'X_test', 'y_test'], None, 'evaluate_model_node')
])

In [101]:
context.project_path

PosixPath('/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflightsRepo')

In [160]:
context.params

{'test_size': 0.2,
 'random_state': 3,
 'features': ['engines',
  'passenger_capacity',
  'crew',
  'd_check_complete',
  'moon_clearance_complete',
  'iata_approved',
  'company_rating',
  'review_scores_rating']}

In [102]:
companies_df = catalog.load("companies")
reviews_df = catalog.load("reviews")
shuttles_df = catalog.load("shuttles")

2021-10-07 14:46:45,659 - kedro.io.data_catalog - INFO - Loading data from `companies` (CSVDataSet)...
2021-10-07 14:46:45,710 - kedro.io.data_catalog - INFO - Loading data from `reviews` (CSVDataSet)...
2021-10-07 14:46:45,780 - kedro.io.data_catalog - INFO - Loading data from `shuttles` (ExcelDataSet)...


/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/kedro/extras/datasets/pandas/excel_dataset.py:168: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.
  return pd.read_excel(fs_file, **self._load_args)
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


In [150]:
companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 77096 non-null  int64  
 1   company_rating     47187 non-null  object 
 2   company_location   57966 non-null  object 
 3   total_fleet_count  77089 non-null  float64
 4   iata_approved      77089 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.9+ MB


In [151]:
companies_df.isnull().sum()

id                       0
company_rating       29909
company_location     19130
total_fleet_count        7
iata_approved            7
dtype: int64

In [105]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   shuttle_id               77096 non-null  int64  
 1   review_scores_rating     55956 non-null  float64
 2   review_scores_comfort    55896 non-null  float64
 3   review_scores_amenities  55909 non-null  float64
 4   review_scores_trip       55833 non-null  float64
 5   review_scores_crew       55902 non-null  float64
 6   review_scores_location   55831 non-null  float64
 7   review_scores_price      55828 non-null  float64
 8   number_of_reviews        77096 non-null  int64  
 9   reviews_per_month        57553 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 5.9 MB


In [106]:
reviews_df.isnull().sum()

shuttle_id                     0
review_scores_rating       21140
review_scores_comfort      21200
review_scores_amenities    21187
review_scores_trip         21263
review_scores_crew         21194
review_scores_location     21265
review_scores_price        21268
number_of_reviews              0
reviews_per_month          19543
dtype: int64

In [107]:
shuttles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       77096 non-null  int64  
 1   shuttle_location         77096 non-null  object 
 2   shuttle_type             77096 non-null  object 
 3   engine_type              77096 non-null  object 
 4   engine_vendor            77096 non-null  object 
 5   engines                  77057 non-null  float64
 6   passenger_capacity       77096 non-null  int64  
 7   cancellation_policy      77096 non-null  object 
 8   crew                     76947 non-null  float64
 9   d_check_complete         77096 non-null  object 
 10  moon_clearance_complete  77096 non-null  object 
 11  price                    77096 non-null  object 
 12  company_id               77096 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 7.6+ MB


In [108]:
shuttles_df.isnull().sum()

id                           0
shuttle_location             0
shuttle_type                 0
engine_type                  0
engine_vendor                0
engines                     39
passenger_capacity           0
cancellation_policy          0
crew                       149
d_check_complete             0
moon_clearance_complete      0
price                        0
company_id                   0
dtype: int64

In [109]:
companies_df.head()

,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,100%,Niue,4.0,f
1,30292,67%,Anguilla,6.0,f
2,19032,67%,Russian Federation,4.0,f
3,8238,91%,Barbados,15.0,t
4,30342,NaN,Sao Tome and Principe,2.0,t


In [110]:
import pandas as pd

In [111]:
def _is_true(x):
    return x == "t"


def _parse_percentage(x):
    x = x.str.replace("%", "")
    x = x.astype(float) / 100
    return x


def _parse_money(x):
    x = x.str.replace("$", "").str.replace(",", "")
    x = x.astype(float)
    return x


def preprocess_companies(companies: pd.DataFrame) -> pd.DataFrame:
    """Preprocesses the data for companies.

    Args:
        companies: Raw data.
    Returns:
        Preprocessed data, with `company_rating` converted to a float and
        `iata_approved` converted to boolean.
    """
    companies["iata_approved"] = _is_true(companies["iata_approved"])
    companies["company_rating"] = _parse_percentage(companies["company_rating"])
    return companies


def preprocess_shuttles(shuttles: pd.DataFrame) -> pd.DataFrame:
    """Preprocesses the data for shuttles.

    Args:
        shuttles: Raw data.
    Returns:
        Preprocessed data, with `price` converted to a float and `d_check_complete`,
        `moon_clearance_complete` converted to boolean.
    """
    shuttles["d_check_complete"] = _is_true(shuttles["d_check_complete"])
    shuttles["moon_clearance_complete"] = _is_true(shuttles["moon_clearance_complete"])
    shuttles["price"] = _parse_money(shuttles["price"])
    return shuttles


def create_model_input_table(
    shuttles: pd.DataFrame, companies: pd.DataFrame, reviews: pd.DataFrame
) -> pd.DataFrame:
    """Combines all data to create a model input table.

    Args:
        shuttles: Preprocessed data for shuttles.
        companies: Preprocessed data for companies.
        reviews: Raw data for reviews.
    Returns:
        Model input table.

    """
    rated_shuttles = shuttles.merge(reviews, left_on="id", right_on="shuttle_id")
    model_input_table = rated_shuttles.merge(
        companies, left_on="company_id", right_on="id"
    )
    model_input_table = model_input_table.dropna()
    return model_input_table


In [112]:
preprocess_companies_df = preprocess_companies(companies_df)

In [113]:
preprocess_shuttles_df = preprocess_shuttles(shuttles_df)

/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_47042/4027071226.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace("$", "").str.replace(",", "")


In [114]:
model_input_table = create_model_input_table(preprocess_shuttles_df,preprocess_companies_df,reviews_df)

In [115]:
model_input_table

,id_x,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,...,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month,id_y,company_rating,company_location,total_fleet_count,iata_approved
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
1,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
2,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
3,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
4,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864307,39094,Russian Federation,Type F5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,10.0,10.0,1,1.00,42904,0.7,Russian Federation,2.0,True
1864357,20330,Uzbekistan,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,False,...,10.0,10.0,10.0,1,1.00,5701,1.0,Costa Rica,1.0,True
1864386,16445,Nicaragua,Type V5,Plasma,ThetaBase Services,1.0,1,flexible,3.0,False,...,10.0,10.0,10.0,3,3.00,13728,1.0,Pakistan,1.0,False
1864448,76469,Bouvet Island (Bouvetoya),Type V5,Quantum,ThetaBase Services,1.0,2,moderate,1.0,False,...,10.0,10.0,10.0,1,1.00,41714,1.0,Lebanon,1.0,False


In [116]:
model_input_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759609 entries, 0 to 1864889
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id_x                     759609 non-null  int64  
 1   shuttle_location         759609 non-null  object 
 2   shuttle_type             759609 non-null  object 
 3   engine_type              759609 non-null  object 
 4   engine_vendor            759609 non-null  object 
 5   engines                  759609 non-null  float64
 6   passenger_capacity       759609 non-null  int64  
 7   cancellation_policy      759609 non-null  object 
 8   crew                     759609 non-null  float64
 9   d_check_complete         759609 non-null  bool   
 10  moon_clearance_complete  759609 non-null  bool   
 11  price                    759609 non-null  float64
 12  company_id               759609 non-null  int64  
 13  shuttle_id               759609 non-null  int64  
 14  rev

In [117]:
y = model_input_table.price
X = model_input_table.drop(['price','id_x','company_id','shuttle_id','id_y'],axis=1)

In [118]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759609 entries, 0 to 1864889
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   shuttle_location         759609 non-null  object 
 1   shuttle_type             759609 non-null  object 
 2   engine_type              759609 non-null  object 
 3   engine_vendor            759609 non-null  object 
 4   engines                  759609 non-null  float64
 5   passenger_capacity       759609 non-null  int64  
 6   cancellation_policy      759609 non-null  object 
 7   crew                     759609 non-null  float64
 8   d_check_complete         759609 non-null  bool   
 9   moon_clearance_complete  759609 non-null  bool   
 10  review_scores_rating     759609 non-null  float64
 11  review_scores_comfort    759609 non-null  float64
 12  review_scores_amenities  759609 non-null  float64
 13  review_scores_trip       759609 non-null  float64
 14  rev

In [119]:
#Convert Bool values to 0-1
def bool_to_int(df):
    bool_col = [col for col in df.columns if df[col].dtypes == 'bool']
    print(bool_col)
    for col in bool_col:
        df[col] = df[col].astype('int64')
    return df

X = bool_to_int(X)

['d_check_complete', 'moon_clearance_complete', 'iata_approved']


In [120]:
cat_cols = [col for col in X.columns if X[col].dtypes == 'object' and X[col].nunique() <= 10]
num_cols = [col for col in X.columns if X[col].dtypes in ('int64','float64')]
print("Categorical columns :",cat_cols)
print("Numerical columns   :",num_cols)

Categorical columns : ['engine_type', 'engine_vendor', 'cancellation_policy']
Numerical columns   : ['engines', 'passenger_capacity', 'crew', 'd_check_complete', 'moon_clearance_complete', 'review_scores_rating', 'review_scores_comfort', 'review_scores_amenities', 'review_scores_trip', 'review_scores_crew', 'review_scores_location', 'review_scores_price', 'number_of_reviews', 'reviews_per_month', 'company_rating', 'total_fleet_count', 'iata_approved']


In [121]:
X.shape

(759609, 23)

In [122]:
X['engine_type'].value_counts()

Plasma     669699
Quantum     83227
Nuclear      6683
Name: engine_type, dtype: int64

In [123]:
X['engine_vendor'].value_counts()

ThetaBase Services                  758528
Banks, Wood and Phillips               797
Warwick Technology Multinational       168
SIT Technology Unlimited                81
MCW Global                              35
Name: engine_vendor, dtype: int64

In [124]:
X['cancellation_policy'].value_counts()

strict      673660
moderate     55883
flexible     30066
Name: cancellation_policy, dtype: int64

In [125]:
X = X[cat_cols + num_cols].copy()

In [126]:
X.shape

(759609, 20)

In [127]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("X_train shape:",X_train.shape)
print("y_train shape:",y_train.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",X_test.shape)

X_train shape: (531726, 20)
y_train shape: (531726,)
X_test shape: (227883, 20)
y_test shape: (227883, 20)


In [132]:
#Preprocessing Numerical Data
num_transformer = Pipeline(steps=[
                                  ('scaler',StandardScaler())
                                  ])

#Preprocessing Categorical Data
cat_transformer = Pipeline(steps=[
                                  ('onehot',OneHotEncoder(handle_unknown='ignore'))
                                  ])		
		


#Bundle Preprocessing steps
preprocessor = ColumnTransformer(transformers=[
                                               ('num',num_transformer,num_cols),
                                               ('cat',cat_transformer,cat_cols)
])


In [133]:
RFregressor = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                       max_depth=300, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

pipeline_RFregressor = Pipeline(steps=[('preprocessor', preprocessor),
                           ('RFregressor', RFregressor)])

In [134]:
pipeline_RFregressor.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['engines',
                                                   'passenger_capacity', 'crew',
                                                   'd_check_complete',
                                                   'moon_clearance_complete',
                                                   'review_scores_rating',
                                                   'review_scores_comfort',
                                                   'review_scores_amenities',
                                                   'review_scores_trip',
                                                   'review_scores_crew',
                                                   'review_scores_location',
 

In [135]:
y_pred = pipeline_RFregressor.predict(X_test)
score = r2_score(y_test, y_pred)

In [136]:
print("RFregressorModel with more features have a coefficient R^2 of %.3f on test data.", score)

RFregressorModel with more features have a coefficient R^2 of %.3f on test data. 0.9877658439158198


In [137]:
importance = RFregressor.feature_importances_

In [138]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.39734
Feature: 1, Score: 0.07087
Feature: 2, Score: 0.02345
Feature: 3, Score: 0.00840
Feature: 4, Score: 0.00000
Feature: 5, Score: 0.04836
Feature: 6, Score: 0.01631
Feature: 7, Score: 0.01303
Feature: 8, Score: 0.01468
Feature: 9, Score: 0.00949
Feature: 10, Score: 0.02592
Feature: 11, Score: 0.03210
Feature: 12, Score: 0.03406
Feature: 13, Score: 0.10515
Feature: 14, Score: 0.00848
Feature: 15, Score: 0.11032
Feature: 16, Score: 0.00467
Feature: 17, Score: 0.00004
Feature: 18, Score: 0.07190
Feature: 19, Score: 0.00017
Feature: 20, Score: 0.00000
Feature: 21, Score: 0.00000
Feature: 22, Score: 0.00000
Feature: 23, Score: 0.00000
Feature: 24, Score: 0.00000
Feature: 25, Score: 0.00130
Feature: 26, Score: 0.00225
Feature: 27, Score: 0.00170


In [90]:
type(importance)

numpy.ndarray